In [1]:
import scipy

N = 100000
x_array = scipy.random.rand(N)
y_array = scipy.random.rand(N)
# generate N pseudorandom independent x and y-values on interval [0,1)
N_qtr_circle = sum(x_array ** 2 + y_array ** 2 < 1)
# Number of pts within the quarter circle x^2 + y^2 < 1 centered at the origin with radius r=1.
# True area of quarter circle is pi/4 and has N_qtr_circle points within it.
# True area of the square is 1 and has N points within it, hence we approximate pi with
pi_approx = 4 * float(N_qtr_circle) / N  # Typical values: 3.13756, 3.15156
pi_approx

AttributeError: Module 'scipy' has no attribute 'random'

In [ ]:
N = 100000
scipy.random.seed(4)
x_array = scipy.random.rand(N)
y_array = scipy.random.rand(N)
# generate N pseudorandom independent x and y-values on interval [0,1)
N_qtr_circle = sum(x_array ** 2 + y_array ** 2 < 1)
# Number of pts within the quarter circle x^2 + y^2 < 1 centered at the origin with radius r=1.
# True area of quarter circle is pi/4 and has N_qtr_circle points within it.
# True area of the square is 1 and has N points within it, hence we approximate pi with
pi_approx = 4 * float(N_qtr_circle) / N  # Typical values: 3.13756, 3.15156
pi_approx

# MCMC


In [ ]:
import math

# Linear Congruential Generator for generating uniform random numbers
class LCG:
    def __init__(self, seed=42):
        self.modulus = 2**32
        self.a = 1664525
        self.c = 1013904223
        self.state = seed

    def rand(self):
        self.state = (self.a * self.state + self.c) % self.modulus
        return self.state / self.modulus

# Gaussian (normal) probability density function
def normal_pdf(x, mean, std):
    return (1 / (std * math.sqrt(2 * math.pi))) * math.exp(-0.5 * ((x - mean) / std) ** 2)

# Metropolis-Hastings algorithm for MCMC sampling
def metropolis_hastings(target_pdf, proposal_std, n_samples, initial_x, mean, std):
    rng = LCG()  # Initialize random number generator
    samples = []
    x = initial_x  # Start point for the chain
    
    for _ in range(n_samples):
        # Propose a new candidate from a Gaussian proposal distribution
        u1, u2 = rng.rand(), rng.rand()
        z1, _ = box_muller_transform(u1, u2)
        candidate_x = x + proposal_std * z1  # Proposal step

        # Acceptance probability (Metropolis-Hastings criterion)
        acceptance_prob = min(1, target_pdf(candidate_x, mean, std) / target_pdf(x, mean, std))
        
        # Accept or reject the candidate
        if rng.rand() < acceptance_prob:
            x = candidate_x  # Accept the candidate
        
        samples.append(x)  # Store the sample

    return samples

# Box-Muller Transform to generate normal distributed numbers (used in the proposal distribution)
def box_muller_transform(u1, u2):
    z1 = math.sqrt(-2 * math.log(u1)) * math.cos(2 * math.pi * u2)
    z2 = math.sqrt(-2 * math.log(u1)) * math.sin(2 * math.pi * u2)
    return z1, z2

# Function to calculate empirical mean and variance
def calculate_statistics(samples):
    n = len(samples)
    mean = sum(samples) / n
    variance = sum((x - mean) ** 2 for x in samples) / n
    return mean, variance

# Parameters for normal distribution (target distribution)
mean_true = 5
std_true = 2
n_samples = 10000
initial_x = 0  # Starting point for the chain
proposal_std = 1.0  # Standard deviation for the proposal distribution

# Run the Metropolis-Hastings MCMC sampler
samples = metropolis_hastings(normal_pdf, proposal_std, n_samples, initial_x, mean_true, std_true)

# Calculate the estimated mean and variance of the samples
mean_estimate, variance_estimate = calculate_statistics(samples)

# Display the results
print(f"True Mean: {mean_true}, Estimated Mean: {mean_estimate}")
print(f"True Variance: {std_true**2}, Estimated Variance: {variance_estimate}")